**Importy**

In [82]:
import regex
from csv import writer
import pandas as pd 

**Premenne**

In [83]:
debug_filepath = "D:/Skola/7.semester/VINF/projekt/data/test_data_1page.xml"
input_filepath = "D:/Skola/7.semester/VINF/projekt/data/skwiki-latest-pages-articles.xml"
output_filepath = "D:/Skola/7.semester/VINF/projekt/data/anchors.csv"

**Funkcie**

In [88]:
def open_read(input_filepath,output_filepath,process):
    with open(input_filepath, encoding='UTF-8') as input_file ,open(output_filepath, 'w+', encoding='UTF-8' ) as output_file:
        csv_writer = writer(output_file)
        csv_writer.writerow(["Link", "Alt_Text"])
        for line in input_file:
            process(line,csv_writer)

In [89]:
def process(line,csv_writer):
    regex_patern = '\[\[([\p{L}\p{Pc}\p{Po}\p{Pi}\p{Pf}\p{Ps}\p{Pd}\p{Sk}\p{So}\p{C}\p{N}\p{Z}})= ]+)\|?([\p{L}\p{Pc}\p{Po}\p{Pi}\p{Pf}\p{Ps}\p{Pd}\p{Sk}\p{So}\p{C}\p{N}\p{Z}})= ]+)?\]\]'
    matches = regex.findall(regex_patern, line)
    for match in matches:
        list_matche = list(filter(None, match))
        
        print(list_matche)
        
        csv_writer.writerow(list_matche)

**Priebeh spracovania**

In [90]:
open_read(debug_filepath,output_filepath,process)

['Šablóna:Hlavná stránka/Aktuality']
['Special:Permalink/7069577#Preview_hlavnej_stránky_s_aktualitami', 'návrhu']
['Wikipédia:O Wikipédii', 'O Wikipédii']
['Wikipédia:Príručka', 'Príručka']
['Pomoc:Obsah', 'Pomoc']
['Wikipédia:Zoznam najlepších článkov', 'Najlepšie články']
[':Kategória:Wikiportály', 'Portály']
[':Kategória:Základné kategórie', 'Kategórie']
['Wikipédia:Rýchly index', 'A{{--}}Z index']
['Súbor:Bluebg.png', 'link=']
['{{CURRENTDAY}}. {{CURRENTMONTHNAME}}']
['Súbor:Bluebg.png', 'link=']
['Súbor:Bluebg.png', 'link=']
['Súbor:Bluebg.png', 'link=']
['Súbor:Bluebg.png', 'link=']
['Hlavná stránka']
['Redaktor:Liso', 'Liso']
[':Obrázok:550px-Solar_system_scale.jpg', '550px-Solar_system_scale.jpg']
['Redaktor:Valasek', 'Valasek']
[':Obrázok:Wiki.png', 'Wiki.png']
['Redaktor:Liso', 'Liso']
[':Obrázok:Go-small.png', 'Go-small.png']
['Redaktor:Ujo Ivo', 'Ujo Ivo']
[':Obrázok:Cad.jpg', 'Cad.jpg']
['Redaktor:Ujo Ivo', 'Ujo Ivo']
[':Obrázok:Boole.jpg', 'Boole.jpg']
['Redaktor:Ujo Ivo

In [87]:

data = pd.read_csv(output_filepath) 
data.head(100)

,Link,Alt_Text
0,Šablóna:Hlavná stránka/Aktuality,NaN
1,Special:Permalink/7069577#Preview_hlavnej_strá...,návrhu
2,Wikipédia:O Wikipédii,O Wikipédii
3,Wikipédia:Príručka,Príručka
4,Pomoc:Obsah,Pomoc
5,Wikipédia:Zoznam najlepších článkov,Najlepšie články
6,:Kategória:Wikiportály,Portály
7,:Kategória:Základné kategórie,Kategórie
8,Wikipédia:Rýchly index,A{{--}}Z index
9,Súbor:Bluebg.png,link=
